In [1]:
from selenium import webdriver
from time import sleep
import json
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup

driver = webdriver.Chrome('./chromedriver')
# 나중에 이거 주소를 리스트에 넣고 하나씩 반복해서 돌리면 될거같음
# 이거 저기 ms부분만 리스트에 넣고 url주소 바꾸면서 가져오면 될거같음 (ms가 위도경도임)
driver.get('https://new.land.naver.com/rooms?ms=37.576192,126.921263,11&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ae=ONEROOM')


In [3]:
sleep(3)

content = BeautifulSoup(driver.page_source, 'html.parser')
# 원룸/투룸 클릭 
driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
sleep(1)
# 투룸 제외(원룸만) -> 이거 체크되어있는지 확인하고(area-pressed = true인지 false인지)
# 체크되어있으면 클릭 안하도록 만들기
# driver.find_element_by_css_selector("#type1").click()
# sleep(1)
driver.find_element_by_css_selector("#list > div > div > div.list_fixed > div > a:nth-child(2)").click()
sleep(1)

In [4]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

actions = ActionChains(driver)

# 페이지 전체를 스크롤다운 하려는게 아니라 옆에 리스트만 스크롤 다운하고 싶기 때문에 이걸 해줌
scrollArea = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
sleep(1)

# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)

# actions.move_to_element(element).perform()
last_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)

try:
    a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
except NoSuchElementException:
    a = None

while(True):
    # 무한스크롤의 끝까지 늘려줌
    # 스크롤이 끝나기 전에는 loader이라는 div가 있는데 그게 더 이상 없을 때까지 스크롤을 내려줌  
    while a != None:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)
        sleep(1)
        try:
            new_a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
        except NoSuchElementException:
            new_a = None
        sleep(1)    
        if new_a != a:
            a = new_a
        
    new_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)
    if new_height == last_height:
        break
    last_height = new_height

print("loading complete")
# new_height = driver.execute_script("arguments[0].scrollTo(0, 0);", scrollArea)
# 스크롤바 제일 위로 올라감
driver.execute_script("arguments[0].scrollTo(0,0);", scrollArea)
sleep(3)

loading complete


In [5]:
# content = BeautifulSoup(driver.page_source, 'html.parser')

# # span class명 정리
# # area is-selected: xx시 xx구 xx동
# # text: 건물명
# # type: 월세/전세를 구분함
# # spec: 공급면적/전용면적, 해당층/총층, 집 방향(ex.남동향 등등)
# # price: 월세/전세의 가격. 월세면 보증금/월세 의 형태로 나타남
# # strong clas명 정리
# # type: 건물 형태(ex.아파트,원룸 등등)

# # em class명 정리
# # title: 확인 = 아직 거래가 되지 않음&확인된 매물
# #        거래완료 = 거래가 완료된 매물
# #        등록일 = 아직 거래가 되지 않음&확인이 되지 않은 매물

# # 나중에 아이템을 하나씩 찾도록 바꾸기

# # list_items = content.findAll('span',{'class':["area is-selected","type","price"]})
# # list_items_a = content.select('#region_filter > div > a > span')
# # list_items_b = []
# # list_items_c = []
# # for item in list_items_a:
# #     list_items_b.append(item.text)
    
# # for item in list_items:
# #     list_items_c.append(item.text)

# city = []
# Gu = []
# Dong = []
# place = []
# list_area = content.select('span.area.is-selected')
# for item in list_area:
#     place.append(item.text)
# city.append(place[0])
# Gu.append(place[1])
# Dong.append(place[2])
# print(place)
    

# # 건물명
# names = []
# list_name = content.select('span.text')
# for item in list_name:
#     names.append(item.text)
    
# # 월세/전세
# types = []
# list_type = content.select('span.type')
# for item in list_type:
#     types.append(item.text)

# # 가격
# prices = []
# list_price = content.select('span.price')
# for item in list_price:
#     prices.append(item.text)
    
# # 건물 형태
# buildings = []
# list_build = content.select('strong.type')
# for item in list_build:
#     buildings.append(item.text)

# # 기타 정보
# specifics = []
# list_spec = content.select('span.spec')
# for item in list_spec:
#     specifics.append(item.text)

# # 판매여부
# checks = []
# list_check = content.select('em.title')
# for item in list_check:
#     checks.append(item.text)

# print(city)
# print(Gu)
# print(Dong)
# print(names)
# print(types)
# print(prices)
# print(buildings)
# print(specifics)
# print(checks)
# # class House:
# #     def __init__(self, payment, price):
# #         self.payment = payment
# #         self.price = price
    
# #     def __str__(self):
# #         return self.payment + " = " + self.price
    
# #     def __repr__(self):
# #         return str(self)
# # print(list_items)    
# # print(list_items_b)
# # print(list_items_c)

['서울시', '서대문구', '연희동']
['서울시']
['서대문구']
['연희동']
['정연지오빌 1동', '일반원룸 ', '일반원룸 ', '도시형생활주택 ', '일반원룸 ', '일반원룸 ', '오피스텔 ', '일반원룸 서일빌 ', '일반원룸 ', '일반원룸 ', '일반원룸 JOY HOUSE ', '일반원룸 ', '정연지오빌 1동', '정연지오빌 1동', '시아루비움 1동', '일반원룸 ', '다가구 백년주택 B동 ', '단독 ', '빌라 ', '정연지오빌 1동', '연희웨스트팰리스(도시형) 1동', '연희미다스파크 104동', '일반원룸 JOY HOUSE ', '일반원룸 JOY HOUSE ', '일반원룸 ', '일반원룸 ', '파라다이스텔SB 1동', '파라다이스텔SB 1동', '일반원룸 ', '기타 ', '일반원룸 원광빌라 ', '일반원룸 ', '연립 ', '일반원룸 ', '더연희힐(도시형) 1동', '일반원룸 ', '일반원룸 ', '일반원룸 그린 하우스 ', '연희브라운스톤(도시형) B동', '연희소프트빌 1동', '일반원룸 ', '일반원룸 ', '오벨리스크 1동', '일반원룸 ', '일반원룸 Time Square ', '일반원룸 영원해피하우스 ', '일반원룸 ', '기타 우리집 ', '단독 ', '스페이스엠 1동', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '오벨리스크 1동', '파라다이스텔SB 1동', '일반원룸 ', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '다가구 ', '정연지오빌 1동', '정연지오빌II 1동', '일반원룸 ', '오피스텔 ', '빌라 ', '연희웨스트팰리스(도시형) 1동', '다가구 ', '다가구 ', '일반원룸 ', '정연지오빌II 1동', '빌라 디타운하우스 ', '일반원룸 Y-HOUSE ', '오벨리스크 1동', '오피스텔 ', '다가구 백년주택 B동 ', '다가구 ', '일반원룸 ', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1

In [5]:
content = BeautifulSoup(driver.page_source, 'html.parser')

# 건물명
names = []
list_name = content.select('#listContents1 > div > div > div > div > div > a.item_link > div.item_title > span.text')
for item in list_name:
    names.append(item.text)

num = len(names)
print(num)

city = []
Gu = []
Dong = []
place = []
list_area = content.select('span.area.is-selected')
for item in list_area:
    place.append(item.text)
city.append(place[0])
Gu.append(place[1])
Dong.append(place[2])

i = 1
p_final = []
while i <= len(names):
    page = "#listContents1 > div > div > div:nth-child(1) > div:nth-child("+str(i)+")"
    p_name = []
    p_type = []
    p_price = []
    p_build = []
    p_spec = []
    p_check = []
    p_date = []
    p_total = []
    
    p_total.append(city[0])
    p_total.append(Gu[0])
    p_total.append(Dong[0])
    # 건물명
    pp_name = content.select(page + " > div > a.item_link > div.item_title > span.text")
    for item in pp_name:
        p_name.append(item.text)
    for item in p_name:
        p_total.append(item)
    
    # 월세/전세
    pp_type = content.select(page + " > div > a.item_link > div.price_line > span.type")
    for item in pp_type:
        p_type.append(item.text)
    for item in p_type:
        p_total.append(item)
    
    # 가격
    pp_price = content.select(page + " > div > a > div.price_line > span.price")
    for item in pp_price:
        p_price.append(item.text)
    for item in p_price:
        p_total.append(item)
        
    # 건물 형태
    pp_build = content.select(page + " > div > a.item_link > div.info_area > p:nth-child(1) > strong")
    for item in pp_build:
        p_build.append(item.text)
    for item in p_build:
        p_total.append(item)

    # 기타 정보
    pp_spec = content.select(page + " > div > a.item_link > div.info_area > p:nth-child(1) > span")
    for item in pp_spec:
        p_spec.append(item.text)
    for item in p_spec:
        p_total.append(item)

    # 판매여부
    pp_check = content.select(page + " > div > div.label_area > span > em.title")
    for item in pp_check:
        p_check.append(item.text)
    for item in p_check:
        p_total.append(item)
        
    # 날짜
    pp_date = content.select(page + " > div > div.label_area > span > em.data")
    for item in pp_date:
        p_date.append(item.text)
    for item in p_date:
        p_total.append(item)


    p_final.append(p_total)
        
    i += 1
    
print(p_final)

# page = "#listContents1 > div > div > div:nth-child(1) > div:nth-child("+str(1)+")"
# p_page = page + " > div > a > div.price_line > span.price"
# print(p_page)
# print(content.select(p_page))
    

68
[['서울시', '서대문구', '남가좌동', '다가구 ', '월세', '500/25', '단독/다가구', '25/19m², B1/2층', '확인', '19.11.28.'], ['서울시', '서대문구', '남가좌동', '일반원룸 ', '월세', '1,000/35', '원룸', '23/23m², 3/4층', '확인', '19.11.28.'], ['서울시', '서대문구', '남가좌동', '일반원룸 월드하우스 ', '월세', '500/50', '원룸', '21/15m², 4/5층, 남향', '확인', '19.11.28.'], ['서울시', '서대문구', '남가좌동', '일반원룸 VILL de UNI ', '월세', '300/30', '원룸', '25/12m², 1/5층, 남동향', '확인', '19.11.28.'], ['서울시', '서대문구', '남가좌동', '다가구 ', '월세', '500/25', '단독/다가구', '18/17m², 3/3층, 남서향', '확인', '19.11.28.'], ['서울시', '서대문구', '남가좌동', '일반원룸 ', '전세', '1억 3,000', '원룸', '23/22m², 3/5층', '확인', '19.11.27.'], ['서울시', '서대문구', '남가좌동', '일반원룸 ', '전세', '7,000', '원룸', '27/17m², 2/4층, 남동향', '확인', '19.11.26.'], ['서울시', '서대문구', '남가좌동', '일반원룸 TAESUNG HOUSE ', '월세', '500/37', '원룸', '30/21m², 2/4층, 남동향', '확인', '19.11.26.'], ['서울시', '서대문구', '남가좌동', '일반원룸 ', '월세', '500/47', '원룸', '30/18m², 4/4층, 남동향', '확인', '19.11.26.'], ['서울시', '서대문구', '남가좌동', '일반원룸 ', '전세', '1억 3,000', '원룸', '29/17m², 3/5층', '확인', '19.11.23.'], ['서

In [6]:
print(p_final)

[['서울시'], ['서대문구'], ['연희동'], ['정연지오빌 1동'], ['월세'], ['1,000/55'], ['오피스텔'], ['25A/19m², 4/6층, 남향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['일반원룸 '], ['월세'], ['300/35'], ['원룸'], ['11/9m², 5/5층, 북동향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['일반원룸 '], ['전세'], ['7,000'], ['원룸'], ['15/13m², 4/5층, 남동향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['도시형생활주택 '], ['월세'], ['1,000/75'], ['원룸'], ['25/13m², 9/10층, 남서향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['정연지오빌 1동'], ['월세'], ['1,000/50'], ['오피스텔'], ['25A/19m², 2/6층, 동향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['연희웨스트팰리스(도시형) 1동'], ['월세'], ['1,000/65'], ['아파트'], ['20/13m², 저/15층, 남향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['일반원룸 '], ['월세'], ['1,000/50'], ['원룸'], ['23/21m², 2/4층'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['일반원룸 '], ['월세'], ['1,000/55'], ['원룸'], ['24/20m², 2/4층, 남향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['오피스텔 '], ['월세'], ['500/50'], ['원룸'], ['24/16m², 3/10층, 남향'], ['확인'], ['서울시'], ['서대문구'], ['연희동'], ['일반원룸 서일빌 '], ['월세'], ['1,000/45'], ['원룸'], ['22/

In [6]:
data = pd.DataFrame(p_final)
data.head()

,0,1,2,3,4,5,6,7,8,9
0,서울시,서대문구,남가좌동,다가구,월세,500/25,단독/다가구,"25/19m², B1/2층",확인,19.11.28.
1,서울시,서대문구,남가좌동,일반원룸,월세,"1,000/35",원룸,"23/23m², 3/4층",확인,19.11.28.
2,서울시,서대문구,남가좌동,일반원룸 월드하우스,월세,500/50,원룸,"21/15m², 4/5층, 남향",확인,19.11.28.
3,서울시,서대문구,남가좌동,일반원룸 VILL de UNI,월세,300/30,원룸,"25/12m², 1/5층, 남동향",확인,19.11.28.
4,서울시,서대문구,남가좌동,다가구,월세,500/25,단독/다가구,"18/17m², 3/3층, 남서향",확인,19.11.28.


In [10]:
data.columns = ['시', '구', '동', '이름', '월세/전세', '가격', '건물형태', '기타정보', '확인/판매여부', '날짜']
data.head()

,시,구,동,이름,월세/전세,가격,건물형태,기타정보,확인/판매여부,날짜
0,서울시,서대문구,남가좌동,다가구,월세,500/25,단독/다가구,"25/19m², B1/2층",확인,19.11.28.
1,서울시,서대문구,남가좌동,일반원룸,월세,"1,000/35",원룸,"23/23m², 3/4층",확인,19.11.28.
2,서울시,서대문구,남가좌동,일반원룸 월드하우스,월세,500/50,원룸,"21/15m², 4/5층, 남향",확인,19.11.28.
3,서울시,서대문구,남가좌동,일반원룸 VILL de UNI,월세,300/30,원룸,"25/12m², 1/5층, 남동향",확인,19.11.28.
4,서울시,서대문구,남가좌동,다가구,월세,500/25,단독/다가구,"18/17m², 3/3층, 남서향",확인,19.11.28.


In [9]:
data.to_csv('testCrawl.csv', encoding='cp949') 

In [11]:
data['공급면적/전용면적'] = data.기타정보.str.split(',').str[0]

In [13]:
del data["기타정보"]

In [15]:
data['보증금'] = data.가격.str.split('/').str[0]
data['월세'] = data.가격.str.split('/').str[1]

In [17]:
print(data[data['월세/전세'].isin(['전세'])]) # '월세, 전세'열에서 '전세를 가지고 있는 행 출력

      시     구     동           이름 월세/전세        가격    건물형태 확인/판매여부         날짜  \
5   서울시  서대문구  남가좌동        일반원룸     전세  1억 3,000      원룸      확인  19.11.27.   
6   서울시  서대문구  남가좌동        일반원룸     전세     7,000      원룸      확인  19.11.26.   
9   서울시  서대문구  남가좌동        일반원룸     전세  1억 3,000      원룸      확인  19.11.23.   
13  서울시  서대문구  남가좌동  일반원룸 노블하우스     전세     5,000      원룸      확인  19.11.22.   
18  서울시  서대문구  남가좌동        일반원룸     전세  1억 3,000      원룸      확인  19.11.20.   
19  서울시  서대문구  남가좌동         다가구     전세  1억 3,000  단독/다가구      확인  19.11.20.   
22  서울시  서대문구  남가좌동        일반원룸     전세  1억 3,000      원룸      확인  19.11.18.   
26  서울시  서대문구  남가좌동          단독     전세     5,000  단독/다가구      확인  19.11.15.   
28  서울시  서대문구  남가좌동        일반원룸     전세     5,500      원룸      확인  19.11.13.   
32  서울시  서대문구  남가좌동        일반원룸     전세     7,000      원룸      확인  19.11.11.   
39  서울시  서대문구  남가좌동        일반원룸     전세     9,500      원룸      확인  19.11.06.   
55  서울시  서대문구  남가좌동        일반원룸     전세     5,000    

In [16]:
data.head()

,시,구,동,이름,월세/전세,가격,건물형태,확인/판매여부,날짜,공급면적/전용면적,보증금,월세
0,서울시,서대문구,남가좌동,다가구,월세,500/25,단독/다가구,확인,19.11.28.,25/19m²,500,25
1,서울시,서대문구,남가좌동,일반원룸,월세,"1,000/35",원룸,확인,19.11.28.,23/23m²,"1,000",35
2,서울시,서대문구,남가좌동,일반원룸 월드하우스,월세,500/50,원룸,확인,19.11.28.,21/15m²,500,50
3,서울시,서대문구,남가좌동,일반원룸 VILL de UNI,월세,300/30,원룸,확인,19.11.28.,25/12m²,300,30
4,서울시,서대문구,남가좌동,다가구,월세,500/25,단독/다가구,확인,19.11.28.,18/17m²,500,25


In [13]:
names = []
list_name = content.select('#listContents1 > div > div > div > div > div > a.item_link > div.item_title > span.text')
for item in list_name:
    names.append(item.text)

num = len(names)
print(num)
print(names)

68
['다가구 ', '일반원룸 ', '일반원룸 월드하우스 ', '일반원룸 VILL de UNI ', '다가구 ', '일반원룸 ', '일반원룸 ', '일반원룸 TAESUNG HOUSE ', '일반원룸 ', '일반원룸 ', 'DMC엘가(도시형) 1동', '일반원룸 노블하우스 ', '일반원룸 노블하우스 ', '일반원룸 노블하우스 ', '일반원룸 노블하우스 ', '일반원룸 노블하우스 ', '빌라 수정빌라 ', '일반원룸 ', '일반원룸 ', '다가구 ', 'DMC엘가(도시형) 1동', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '빌라 ', '단독 ', '일반원룸 수현 원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '다가구 ', 'DMC엘가(도시형) 1동', 'DMC엘가(도시형) 1동', 'DMC엘가(도시형) 1동', '일반원룸 ', '일반원룸 ', '오피스텔 ', '빌라 ', 'DMC엘가(도시형) 1동', '일반원룸 ', '일반원룸 미르빌 ', 'DMC엘가(도시형) 1동', 'DMC엘가(도시형) 1동', 'DMC엘가(도시형) 1동', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '빌라 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 ', '일반원룸 두레마을 ', '일반원룸 ', '다가구 ', '일반원룸 ', '일반원룸 ', '일반원룸 DreamCastle I  ', '일반원룸 ', '기타 수정빌라 ', '일반원룸 ']


In [21]:
items = []
for item in list_items:
    item_payment = item.type
    item_price = item.price
    
    if item_payment is not None and item_price is not None:
        new_item = House(item_payment.text, item_price.text)
        items.append(new_item)
        
print(items)
    

[]


In [7]:
house = []
for item in range(len(list_items_a)): 
    new = House(list_items_a[item], list_items_b[item])
    house.append(new)

print(house)

TypeError: unsupported operand type(s) for +: 'Tag' and 'str'